In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from copy import deepcopy

games_csv_path = '/kaggle/input/nfl-big-data-bowl-2024/'

# Load the additional required datasets
plays_df = pd.read_csv(games_csv_path +'plays.csv')
players_df = pd.read_csv(games_csv_path +'players.csv')
games_df = pd.read_csv(games_csv_path +'games.csv')
tackles_df = pd.read_csv(games_csv_path +'tackles.csv')

def calculate_regression_stats(x_data, y_data):
    """
    Calculate both the correlation coefficient and the R-squared value for a linear fit.
    
    Parameters:
    x_data (iterable): The x-values of the data points.
    y_data (iterable): The y-values of the data points.
    
    Returns:
    tuple: A tuple containing the correlation coefficient and the R-squared value.
    """
    slope, intercept, r_value, p_value, std_err = stats.linregress(x_data, y_data)
    r_squared = r_value ** 2
    return r_value, r_squared

team_colors = {
    'ARI': '#97233F',  # Arizona Cardinals
    'ATL': '#A71930',  # Atlanta Falcons
    'BAL': '#241773',  # Baltimore Ravens
    'BUF': '#00338D',  # Buffalo Bills
    'CAR': '#0085CA',  # Carolina Panthers
    'CHI': '#C83803',  # Chicago Bears
    'CIN': '#FB4F14',  # Cincinnati Bengals
    'CLE': '#FF3C00',  # Cleveland Browns
    'DAL': '#041E42',  # Dallas Cowboys
    'DEN': '#FB4F14',  # Denver Broncos
    'DET': '#0076B6',  # Detroit Lions
    'GB':  '#203731',  # Green Bay Packers
    'HOU': '#03202F',  # Houston Texans
    'IND': '#002C5F',  # Indianapolis Colts
    'JAX': '#006778',  # Jacksonville Jaguars
    'KC':  '#E31837',  # Kansas City Chiefs
    'LA':  '#003594',  # Los Angeles Rams
    'LAC': '#0080C6',  # Los Angeles Chargers
    'LV':  '#A5ACAF',  # Las Vegas Raiders
    'MIA': '#008E97',  # Miami Dolphins
    'MIN': '#4F2683',  # Minnesota Vikings
    'NE':  '#002244',  # New England Patriots
    'NO':  '#D3BC8D',  # New Orleans Saints
    'NYG': '#0B2265',  # New York Giants
    'NYJ': '#125740',  # New York Jets
    'PHI': '#004C54',  # Philadelphia Eagles
    'PIT': '#FFB612',  # Pittsburgh Steelers
    'SEA': '#002244',  # Seattle Seahawks
    'SF':  '#AA0000',  # San Francisco 49ers
    'TB':  '#D50A0A',  # Tampa Bay Buccaneers
    'TEN': '#0C2340',  # Tennessee Titans
    'WAS': '#773141',  # Washington Commanders
}

# Calculating a score for each play based on tackles and assists
# Base score is 0, each tackle is 1 point, each assist is 0.5 points
# Total score for a play is capped at 1 point

# Calculating individual player scores within each play
tackles_df['player_score'] = tackles_df['tackle'] + 0.5 * tackles_df['assist']

# Grouping by gameId and playId to calculate total score for each play
play_scores = tackles_df.groupby(['gameId', 'playId'])['player_score'].sum().reset_index()
play_scores['total_score'] = play_scores['player_score'].apply(lambda x: min(x, 1))
play_scores = play_scores.drop('player_score', axis=1)

play_scores_with_team = play_scores.merge(plays_df[['gameId', 'playId', 'defensiveTeam']], on=['gameId', 'playId'], how='left')

# Grouping by defensiveTeam to calculate total points for each team
team_total_scores = play_scores_with_team.groupby('defensiveTeam')['total_score'].sum().reset_index()
team_total_scores.columns = ['Team', 'Total Tackle Play Score']
team_total_scores = team_total_scores.sort_values(by='Total Tackle Play Score', ascending=False)

home_games = games_df.groupby('homeTeamAbbr').size().rename('games')
visitor_games = games_df.groupby('visitorTeamAbbr').size().rename('games')
total_games = home_games.add(visitor_games, fill_value=0)
# Merging the total games count with the team total scores
team_scores_games = team_total_scores.merge(total_games, left_on='Team', right_index=True)

# Calculating the average 'Tackle Play Score' per game for each team
team_scores_games['Average Tackle Play Score per Game'] = team_scores_games['Total Tackle Play Score'] / team_scores_games['games']

# Counting wins for home and visiting teams
home_wins = games_df[games_df['homeFinalScore'] > games_df['visitorFinalScore']].groupby('homeTeamAbbr').size()
visitor_wins = games_df[games_df['visitorFinalScore'] > games_df['homeFinalScore']].groupby('visitorTeamAbbr').size()

total_wins = home_wins.add(visitor_wins, fill_value=0).rename('Wins')
games_played = total_games

# Creating a DataFrame with wins and games played
team_performance = pd.DataFrame({'Wins': total_wins, 'Games Played': games_played})
team_performance['Win Percentage'] = team_performance['Wins'] / team_performance['Games Played']
team_performance = team_performance.reset_index().rename(columns={'index': 'Team'})

# Exploring Tackles in the NFL: An Analytical Approach

Welcome to my analysis for the NFL Big Data Bowl 2024.

In this initial stage, I am focusing on two key aspects of the game: the win rates and the number of tackles per game, analyzed on a team-by-team basis. My objective is to develop an initial understanding of how these metrics correlate with team performance and strategies.

The driving force behind this exploration is to uncover patterns and insights that might explain the dynamics of success in the NFL. As I delve into the data, my goal is to unearth findings that not only enrich my understanding of the sport but also provide valuable insights for teams, strategists, and football enthusiasts.

Join me on this journey of data exploration in the captivating world of the NFL.


## Initial Analysis: High Win Rates and Tackle Influence

Let's take a quick glance at the teams that boasted the highest win rates during the first half of the last NFL season. We'll also explore how the win rates correlate with their tackling statistics.

In [ ]:
# Sorting the team_performance DataFrame by 'Win Percentage' in descending order to rank the teams
ranked_team_performance = team_performance.sort_values(by='Win Percentage', ascending=False)
ranked_team_performance = ranked_team_performance.reset_index(drop=True)

ranked_team_performance[['Team', 'Win Percentage']].head(10)

In [ ]:
merged_data = team_performance.merge(team_scores_games[['Team', 'Average Tackle Play Score per Game']], on='Team')
merged_data['Team Color'] = merged_data['Team'].map(team_colors)

In [ ]:
plt.figure(figsize=(14, 9))

sns.regplot(x='Average Tackle Play Score per Game', y='Win Percentage', data=merged_data)

for team in merged_data['Team'].unique():
    team_data = merged_data[merged_data['Team'] == team]
    plt.scatter(team_data['Average Tackle Play Score per Game'], team_data['Win Percentage'], 
                color=team_data['Team Color'].iloc[0], label=team)

for _, row in merged_data.iterrows():
    if row['Team'] in ['PHI', 'BUF', 'KC', 'MIN', 'NYG']:
        plt.annotate(row['Team'], 
                     (row['Average Tackle Play Score per Game'], row['Win Percentage']),
                     textcoords="offset points", # Positioning label
                     xytext=(15,-5),             # Distance from text to points (x,y)
                     ha='center')              

plt.xlabel('Average # of Tackle Plays per Game')
plt.ylabel('Win Percentage')
plt.title('Win Percentage vs Average Tackle Plays (with Linear Fit)')
plt.legend(title='Teams', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

**Two key observations:**

1. **Lack of Strong Linear Correlation:** The data points do not show a strong linear correlation between win rates and the number of tackle plays. The distribution of points appears somewhat dispersed, indicating that the relationship between these two variables might not be straightforward.

2. **Trend in the Linear Fit:** Despite the lack of a strong correlation, the linear fit suggests a general trend where the win rate tends to decline as the number of tackle plays increases. This trend aligns with the intuitive understanding that more tackles might indicate more defensive plays, which could correlate with lower win rates.

In addition to analyzing win rates, we will delve into the "**Performance Index**," a metric that evaluates a team's scoring efficiency in each game. This index is calculated as the ratio of the team's score to the total points scored in the game, providing a nuanced view of how effectively a team capitalizes on its scoring opportunities. For a given team in a specific game, the Performance Index is computed as:

$$
\text{Performance Index} = \frac{\text{Team's Score}}{\text{Team's Score} + \text{Opponent's Score}}
$$

This index ranges from 0 to 1, where a higher value indicates a higher proportion of points scored by the team. It provides a normalized measure of how effectively a team converted opportunities into points, relative to the overall scoring in the game.


In [ ]:
# Calculate the performance index for each game
games_df['homeTeamIndex'] = games_df['homeFinalScore'] / (games_df['homeFinalScore'] + games_df['visitorFinalScore'])
games_df['visitorTeamIndex'] = games_df['visitorFinalScore'] / (games_df['homeFinalScore'] + games_df['visitorFinalScore'])
games_df.fillna(0, inplace=True)

home_performance = games_df[['homeTeamAbbr', 'homeTeamIndex']].rename(columns={'homeTeamAbbr': 'Team', 'homeTeamIndex': 'PerformanceIndex'})
visitor_performance = games_df[['visitorTeamAbbr', 'visitorTeamIndex']].rename(columns={'visitorTeamAbbr': 'Team', 'visitorTeamIndex': 'PerformanceIndex'})

combined_performance = pd.concat([home_performance, visitor_performance])
average_performance_index = combined_performance.groupby('Team')['PerformanceIndex'].mean().sort_values(ascending=False)

# Plotting
plt.figure(figsize=(15, 8))
average_performance_index.plot(kind='bar')
plt.xlabel('Team')
plt.ylabel('Average Performance Index')
plt.title('Average Performance Index for Each Team')
plt.xticks(rotation=90)
plt.show()

In [ ]:
average_performance_df = average_performance_index.reset_index()
average_performance_df.columns = ['Team', 'Average Performance Index']

merged_data = merged_data.merge(average_performance_df, on='Team', how='left')

Let's take a quick look at how the Performance Index correlates with the tackling statistics.

In [ ]:
plt.figure(figsize=(14, 9))

sns.regplot(x='Average Tackle Play Score per Game', y='Average Performance Index', data=merged_data)

for team in merged_data['Team'].unique():
    team_data = merged_data[merged_data['Team'] == team]
    plt.scatter(team_data['Average Tackle Play Score per Game'], team_data['Average Performance Index'], 
                color=team_data['Team Color'].iloc[0], label=team)

for _, row in merged_data.iterrows():
    if row['Team'] in ['PHI', 'BUF', 'KC', 'MIN', 'NYG']:
        plt.annotate(row['Team'], 
                     (row['Average Tackle Play Score per Game'], row['Average Performance Index']),
                     textcoords="offset points", 
                     xytext=(15, -5), 
                     ha='center')

plt.xlabel('Average # of Tackle Plays per Game')
plt.ylabel('Average Performance Index')
plt.title('Average Performance Index vs Average Tackle Plays (with Linear Fit)')
plt.ylim(0.2, 0.8)
plt.legend(title='Teams', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

By utilizing the Performance Index, the scatter plot exhibits more compact data points. Let's see if we can gain better insights by examining other tackle-related metrics.

In [ ]:
###############

# Analyzing the Impact of Defensive Tackles on EPA

One metric to explore is the "Expected Points Added" (EPA) resulting from tackle plays by the defensive team. Let's take a quick look at the distribution of this key statistic.


In [ ]:
# Clean plays_df to include only plays that are also in tackles_df
plays_df_cleaned = plays_df[plays_df.set_index(['gameId', 'playId']).index.isin(tackles_df.set_index(['gameId', 'playId']).index)]

tackles_enhanced = tackles_df.merge(plays_df_cleaned[['gameId', 'playId', 'quarter', 'down']], on=['gameId', 'playId'])
tackles_enhanced = tackles_enhanced.merge(players_df[['nflId', 'position']], on='nflId', suffixes=('', '_defensivePlayer'))

In [ ]:
# Setting the aesthetic style of the plots
sns.set_style("whitegrid")

# Plotting the distribution of Expected Points Added (EPA)
plt.figure(figsize=(10, 6))
sns.histplot(plays_df_cleaned['expectedPointsAdded'], bins=150, binrange=(-7.5, 7.5), kde=True)
plt.title('Distribution of Expected Points Added (EPA)')
plt.xlabel('Expected Points Added (EPA)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Counting the number of plays for each pass result (C, R, NaN) and calculating average EPA for each
pass_result_counts = plays_df_cleaned['passResult'].value_counts(dropna=False)
average_epa_by_pass_result = plays_df_cleaned.groupby('passResult')['expectedPointsAdded'].mean()

# Including the NaN (designed runs) in the calculation
average_epa_designed_runs = plays_df_cleaned[plays_df_cleaned['passResult'].isna()]['expectedPointsAdded'].mean()

result_df = pd.DataFrame({
    'Number of Plays': pass_result_counts,
    'Average EPA': average_epa_by_pass_result
})

result_df.iloc[0, result_df.columns.get_loc('Average EPA')] = average_epa_designed_runs

When we categorize plays by the "passResult," three distinct types emerge: NaN (Run), Completed (C), and Scramble (R). The mean Expected Points Added (EPA) for each type of play shows notable differences. Below is a table summarizing the number of plays and their corresponding average EPA:

| passResult     | Number of Plays | Average EPA |
|----------------|-----------------|-------------|
| NaN (Run)      | 6215            | -0.088942   |
| C (Completed)  | 5354            | 0.623631    |
| R (Scramble)   | 456             | 0.382322    |

This table offers an insightful look into how different play types contribute to the overall game dynamics in terms of EPA:

1. **Runs (NaN):** With an average EPA of -0.088942, this indicates that tackles in defensive tactics against running plays are often effective, as a negative EPA is beneficial for the defense.

2. **Completed Passes (C):** The high average EPA of 0.623631 for completed passes suggests that these plays are generally advantageous for the offense, leading to positive outcomes in terms of expected points.

3. **Scrambles (R):** Scrambles have an average EPA of 0.382322, which is lower than completed passes but still positive, indicating these plays can be moderately effective for the offense.

The varying EPA values across different play types reflect the effectiveness of defensive strategies, particularly the success of tackles in limiting offensive gains, especially in running plays.


In [ ]:
# Creating a deep copy of the plays_df_cleaned DataFrame
plays_df_scaled = deepcopy(plays_df_cleaned)

# Defining the conditions for each play type
conditions = {
    'C': plays_df_cleaned['passResult'] == 'C',
    'R': plays_df_cleaned['passResult'] == 'R',
    'NaN': plays_df_cleaned['passResult'].isna()
}

# Computing scaling parameters for each play type
scaling_params = {}
for play_type, condition in conditions.items():
    subset = plays_df_cleaned[condition]['expectedPointsAdded']
    scaling_params[play_type] = {
        'mean': subset.mean(),
        'std': subset.std(),
        'min': subset.min(),
        'max': subset.max()
    }

# Applying the scaling parameters to the dataset
for play_type, params in scaling_params.items():
    # Normalization
    plays_df_scaled.loc[conditions[play_type], 'epa_normalized'] = (
        plays_df_cleaned[conditions[play_type]]['expectedPointsAdded'] - params['min']
    ) / (params['max'] - params['min'])
    
    # Standardization
    plays_df_scaled.loc[conditions[play_type], 'epa_standardized'] = (
        plays_df_cleaned[conditions[play_type]]['expectedPointsAdded'] - params['mean']
    ) / params['std']

### Standardization and Normalization

Let's calculate the normalized and standardized values of the Expected Points Added (EPA) for each play, scaled according to the play type (Complete Passes, Scrambles, Designed Runs).

**Standardization** and **Normalization** are two common techniques in data preprocessing to transform data into a more suitable format for analysis.

1. **Standardization**:
   - This technique involves rescaling the distribution of values so that the mean of observed values is 0 and the standard deviation is 1. 
   - Formula: $$ \text{Standardized Value} = \frac{\text{Value} - \text{Mean}}{\text{Standard Deviation}} $$

2. **Normalization**:
   - Normalization rescales the range of features to scale the range in [0, 1] or [-1, 1].
   - Formula: $$ \text{Normalized Value} = \frac{\text{Value} - \text{Min Value}}{\text{Max Value} - \text{Min Value}} $$

The scaling was applied using vectorized operations for efficiency, with separate scaling parameters (mean, standard deviation, min, max) computed for each play type beforehand. This approach ensures that the EPA is adjusted in a contextually relevant manner, making the values more comparable across different play types.

In [ ]:
# Creating a histogram for the distribution of normalized EPA values
plt.figure(figsize=(12, 6))
plt.hist(plays_df_scaled['epa_normalized'].dropna(), bins=50, color='purple', edgecolor='black')
plt.title('Distribution of Normalized EPA')
plt.xlabel('Normalized EPA')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Creating a histogram for the distribution of standardized EPA values for all plays
plt.figure(figsize=(12, 6))
plt.hist(plays_df_scaled['epa_standardized'].dropna(), bins=50, color='orange', edgecolor='black')
plt.title('Distribution of Standardized EPA for All Plays')
plt.xlabel('Standardized EPA')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Merging the required columns from plays_df_scaled to tackles_df based on gameId and playId
tackles_df = pd.merge(tackles_df, plays_df_scaled[['gameId', 'playId', 'epa_standardized', 'epa_normalized']], 
                      on=['gameId', 'playId'], how='left')

# Calculating points for tackles and assists
# Points are equal to 'epa_standardized' for a tackle, and half the value for an assist
tackles_df['points_standardized'] = tackles_df['tackle'] * tackles_df['epa_standardized'] + \
                       (tackles_df['assist'] * tackles_df['epa_standardized']) / 2

tackles_df['points_normalized'] = tackles_df['tackle'] * tackles_df['epa_normalized'] + \
                       (tackles_df['assist'] * tackles_df['epa_normalized']) / 2

# Merge the two dataframes on 'gameId' and 'playId'
merged_df = pd.merge(tackles_df, plays_df_scaled[['gameId', 'playId', 'defensiveTeam']], on=['gameId', 'playId'], how='left')

# Group by 'defensiveTeam' and 'gameId' and 'playId' to ensure each play is only counted once
grouped_df = merged_df.groupby(['defensiveTeam', 'gameId', 'playId']).agg({
    'epa_standardized': 'mean',
    'epa_normalized': 'mean'
}).reset_index()

# Sum the 'epa_standardized' and 'epa_normalized' by team
team_epa_df = grouped_df.groupby('defensiveTeam').agg({
    'epa_standardized': 'sum',
    'epa_normalized': 'sum'
}).reset_index()

# Calculate the number of games played by each team and their win rate
# Create a dictionary to hold the win counts and game counts
team_games = {}
for index, row in games_df.iterrows():
    home_team = row['homeTeamAbbr']
    visitor_team = row['visitorTeamAbbr']
    home_win = row['homeFinalScore'] > row['visitorFinalScore']

    # Update counts for home team
    if home_team not in team_games:
        team_games[home_team] = {'games': 0, 'wins': 0}
    team_games[home_team]['games'] += 1
    if home_win:
        team_games[home_team]['wins'] += 1

    # Update counts for visitor team
    if visitor_team not in team_games:
        team_games[visitor_team] = {'games': 0, 'wins': 0}
    team_games[visitor_team]['games'] += 1
    if not home_win:
        team_games[visitor_team]['wins'] += 1

team_games_df = pd.DataFrame.from_dict(team_games, orient='index').reset_index()
team_games_df.rename(columns={'index': 'defensiveTeam', 'games': 'games_played', 'wins': 'wins'}, inplace=True)
team_games_df['win_rate'] = team_games_df['wins'] / team_games_df['games_played']

team_epa_games_df = pd.merge(team_epa_df, team_games_df, on='defensiveTeam', how='left')

### Total EPA(Scaled)

To further understand the impact of defensive tackles, the sum of the scaled EPA generated by these plays for each team has been calculated. For a more comprehensive analysis, the average value per game will be considered. Below is a table showcasing this data for a selection of teams, along with their games played, wins, and win rate:

| Defensive Team | Total EPA Standardized | Total EPA Normalized | Games Played | Wins | Win Rate  |
|----------------|------------------------|----------------------|--------------|------|-----------|
| ARI            | -8.039853              | 243.382918           | 9            | 3    | 0.333333  |
| ATL            | 3.742326               | 260.263962           | 9            | 4    | 0.444444  |
| BAL            | -5.033219              | 221.922163           | 9            | 6    | 0.666667  |
| BUF            | -15.729452             | 202.712839           | 8            | 6    | 0.750000  |
| CAR            | -15.826527             | 262.079248           | 9            | 2    | 0.222222  |

This table provides an initial overview of the correlation between the EPA contributed by defensive plays and overall team performance in terms of win rate.

**Note**: For both **EPA Standardized** and **EPA Normalized**, the **smaller the values**, the **better** the team performs.


In [ ]:
# Calculate the average scaled epa per game for both scaling methods
team_epa_games_df['avg_epa_standardized_per_game'] = team_epa_games_df['epa_standardized'] / team_epa_games_df['games_played']
team_epa_games_df['avg_epa_normalized_per_game'] = team_epa_games_df['epa_normalized'] / team_epa_games_df['games_played']

merged_data_subset = merged_data[['Team', 'Average Tackle Play Score per Game', 'Team Color', 'Average Performance Index']]
new_merged_df = team_epa_games_df.merge(merged_data_subset, left_on='defensiveTeam', right_on='Team')

new_merged_df.drop('Team', axis=1, inplace=True)
new_merged_df.to_csv('/kaggle/working/scaled_epa.csv', index=False)

Below is a table showing the average values per game for both standardized and normalized EPA for a selection of teams. This data will be used in subsequent plots to explore the relationship between these defensive performance metrics and team success, measured by win rate and performance index:

| Defensive Team | Avg EPA Standardized/Game | Avg EPA Normalized/Game |
|----------------|---------------------------|------------------------|
| ARI            | -0.893317                 | 27.042546              |
| ATL            | 0.415814                  | 28.918218              |
| BAL            | -0.559247                 | 24.658018              |
| BUF            | -1.966181                 | 25.339105              |
| CAR            | -1.758503                 | 29.119916              |

In [ ]:
plt.figure(figsize=(14, 9))

# Regression plot for standardized EPA vs Average Performance Index
sns.regplot(x='avg_epa_standardized_per_game', y='Average Performance Index', data=new_merged_df, scatter=False)

# Scatter plot for each team with standardized EPA
for index, row in new_merged_df.iterrows():
    plt.scatter(row['avg_epa_standardized_per_game'], row['Average Performance Index'], 
                color=row['Team Color'], marker='o', label=row['defensiveTeam'])

# Adding labels for specific teams if necessary
highlight_teams = ['PHI', 'BUF', 'KC', 'MIN', 'NYG']  # Example teams to highlight
for index, row in new_merged_df.iterrows():
    if row['defensiveTeam'] in highlight_teams:
        plt.annotate(row['defensiveTeam'], 
                     (row['avg_epa_standardized_per_game'], row['Average Performance Index']),
                     textcoords="offset points", 
                     xytext=(15, -5), 
                     ha='center')

# Setting up plot titles and labels
plt.title('Average Performance Index vs Average Standardized EPA per Game')
plt.xlabel('Average Standardized EPA per Game')
plt.ylabel('Average Performance Index')
plt.grid(True)
plt.legend(title='Teams', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
plt.figure(figsize=(14, 9))

# Regression plot for normalized EPA vs Average Performance Index
sns.regplot(x='avg_epa_normalized_per_game', y='Average Performance Index', data=new_merged_df, scatter=False)

# Scatter plot for each team with normalized EPA
for index, row in new_merged_df.iterrows():
    plt.scatter(row['avg_epa_normalized_per_game'], row['Average Performance Index'], 
                color=row['Team Color'], marker='o', label=row['defensiveTeam'])

# Adding labels for specific teams if necessary
highlight_teams = ['PHI', 'BUF', 'KC', 'MIN', 'NYG']  # Example teams to highlight
for index, row in new_merged_df.iterrows():
    if row['defensiveTeam'] in highlight_teams:
        plt.annotate(row['defensiveTeam'], 
                     (row['avg_epa_normalized_per_game'], row['Average Performance Index']),
                     textcoords="offset points", 
                     xytext=(15, -5), 
                     ha='center')

# Setting up plot titles and labels
plt.title('Average Performance Index vs Average Normalized EPA per Game')
plt.xlabel('Average Normalized EPA per Game')
plt.ylabel('Average Performance Index')
plt.grid(True)
plt.legend(title='Teams', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
plt.figure(figsize=(14, 9))

sns.regplot(x='avg_epa_standardized_per_game', y='win_rate', data=new_merged_df, scatter=False)

# Scatter plot for each team
for index, row in new_merged_df.iterrows():
    plt.scatter(row['avg_epa_standardized_per_game'], row['win_rate'], 
                color=row['Team Color'], marker='o', label=row['defensiveTeam'])

# Adding labels for specific teams if necessary
highlight_teams = ['PHI', 'BUF', 'KC', 'MIN', 'NYG']  # Example teams to highlight
for index, row in new_merged_df.iterrows():
    if row['defensiveTeam'] in highlight_teams:
        plt.annotate(row['defensiveTeam'], 
                     (row['avg_epa_standardized_per_game'], row['win_rate']),
                     textcoords="offset points", 
                     xytext=(15, -5), 
                     ha='center')

# Setting up plot titles and labels
plt.title('Win Rate vs Average Standardized EPA per Game')
plt.xlabel('Average Standardized EPA per Game')
plt.ylabel('Win Rate')
plt.grid(True)
plt.legend(title='Teams', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
plt.figure(figsize=(14, 9))

# Regression plot for normalized EPA
sns.regplot(x='avg_epa_normalized_per_game', y='win_rate', data=new_merged_df, scatter=False)

# Scatter plot for each team with normalized EPA
for index, row in new_merged_df.iterrows():
    plt.scatter(row['avg_epa_normalized_per_game'], row['win_rate'], 
                color=row['Team Color'], marker='o', label=row['defensiveTeam'])

# Adding labels for specific teams if necessary
highlight_teams = ['PHI', 'BUF', 'KC', 'MIN', 'NYG']  # Example teams to highlight
for index, row in new_merged_df.iterrows():
    if row['defensiveTeam'] in highlight_teams:
        plt.annotate(row['defensiveTeam'], 
                     (row['avg_epa_normalized_per_game'], row['win_rate']),
                     textcoords="offset points", 
                     xytext=(15, -5), 
                     ha='center')

# Setting up plot titles and labels
plt.title('Win Rate vs Average Normalized EPA per Game')
plt.xlabel('Average Normalized EPA per Game')
plt.ylabel('Win Rate')
plt.grid(True)
plt.legend(title='Teams', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
r_value_standardized_performance, r_squared_standardized_performance = calculate_regression_stats(new_merged_df['avg_epa_standardized_per_game'], new_merged_df['Average Performance Index'])
r_value_normalized_performance, r_squared_normalized_performance = calculate_regression_stats(new_merged_df['avg_epa_normalized_per_game'], new_merged_df['Average Performance Index'])
r_value_standardized_winrate, r_squared_standardized_winrate = calculate_regression_stats(new_merged_df['avg_epa_standardized_per_game'], new_merged_df['win_rate'])
r_value_normalized_winrate, r_squared_normalized_winrate = calculate_regression_stats(new_merged_df['avg_epa_normalized_per_game'], new_merged_df['win_rate'])

# Print each set of r_value and r_squared
##print("Standardized EPA vs Average Performance Index: R Value =", r_value_standardized_performance, ", R Squared =", r_squared_standardized_performance)
##print("Normalized EPA vs Average Performance Index: R Value =", r_value_normalized_performance, ", R Squared =", r_squared_normalized_performance)
##print("Standardized EPA vs Win Rate: R Value =", r_value_standardized_winrate, ", R Squared =", r_squared_standardized_winrate)
##print("Normalized EPA vs Win Rate: R Value =", r_value_normalized_winrate, ", R Squared =", r_squared_normalized_winrate)

# Conclusion
## Summary of Correlations

Some intriguing correlations between EPA metrics and team performance indicators. Below is a summarized table of these correlations:

| Correlation Type                                   | R Value        | R Squared     |
|----------------------------------------------------|----------------|---------------|
| Standardized EPA vs Average Performance Index      | -0.203         | 0.0412        |
| Normalized EPA vs Average Performance Index        | -0.324         | 0.105         |
| Standardized EPA vs Win Rate                       | -0.249         | 0.0618        |
| Normalized EPA vs Win Rate                         | -0.237         | 0.0562        |

The relationship between Normalized EPA and the Average Performance Index shows a moderate correlation. This is noteworthy, especially considering that the analysis is based solely on tackle-related statistics. The corresponding scatter plot reveals that even with slight variations in Normalized EPA, there's a noticeable impact on team performance. The decline in team performance as EPA increases suggests a potential area of focus for NFL teams.

In addition to team performance, the scaled EPA metric can be used to assess individual player performance, providing insights into the effectiveness of each player's contributions on the field.

This information can be particularly useful for teams to monitor and assess their tackling efficiency. Since the points are normalized, they provide a more equitable comparison across different tackle types.

Additionally, there's potential for a machine learning approach to further refine the scaling of EPA generated by tackles. While this aspect wasn't completed in this analysis, it represents a promising direction for future exploration to enhance our understanding of tackle efficiency and its impact on game outcomes.


---

**End of Main Report**

Thank you for your attention to this analysis. For additional insights and visualizations, please refer to the appendices.

---


# Acknowledgements

I would like to extend my sincere thanks to the organizers of the NFL Big Data Bowl for creating this fantastic opportunity for data enthusiasts and football fans alike. The chance to dive into such a rich dataset and explore the intricacies of NFL gameplay has been both challenging and immensely rewarding.

While time constraints limited the extent of my analysis, the experience was thoroughly enjoyable and enlightening. I am grateful for having had the opportunity to contribute and learn through this process.

Looking forward to participating in future events, I am excited to continue this journey of data exploration and sports analytics. A big thank you once again to everyone involved in making this event possible and so engaging.


# Appendix

In this appendix, I've included a series of plots and animations that complement the main analysis of the report. These visual elements are designed to provide further insights into the data and to offer an engaging way to explore the dynamics of NFL games.

### Visualizations

- **Individual Defensive Player Tackle Statistics:** This section presents tables and plots showcasing the scaled EPA (Expected Points Added) for individual defensive players. These visualizations in the appendix offer a supplementary view, focusing on the contribution of each player to the team’s defensive performance.
- **Heat Maps:** These offer a visual representation of the density of data, highlighting areas of high activity or interest in the dataset.

In [ ]:
# Calculate the total number of attempts (tackle, assist, or missed tackle) for each player
tackles_df['total_attempts'] = tackles_df['tackle'] + tackles_df['assist'] + tackles_df['pff_missedTackle']

# Summing the 'points_standardized' and 'points_normalized' for each player
# Also calculating the average points per attempt
player_points_summary = tackles_df.groupby('nflId').agg(
    total_points_standardized=pd.NamedAgg(column='points_standardized', aggfunc='sum'),
    total_points_normalized=pd.NamedAgg(column='points_normalized', aggfunc='sum'),
    total_attempts=pd.NamedAgg(column='total_attempts', aggfunc='sum')
)

# Calculate average points per attempt
player_points_summary['average_points_standardized'] = player_points_summary['total_points_standardized'] / player_points_summary['total_attempts']
player_points_summary['average_points_normalized'] = player_points_summary['total_points_normalized'] / player_points_summary['total_attempts']

player_points_summary.reset_index(inplace=True)

In [ ]:
# Merge player_points_summary with players_df to add player names
# We use nflId as the key for merging
player_points_summary = player_points_summary.merge(players_df[['nflId', 'displayName']], on='nflId', how='left')

player_team_info = plays_df_cleaned[['gameId', 'playId', 'defensiveTeam']].drop_duplicates()
tackles_with_team = tackles_df.merge(player_team_info, on=['gameId', 'playId'], how='left')

player_points_summary_with_team = player_points_summary.merge(
    tackles_with_team[['nflId', 'defensiveTeam']].drop_duplicates(), on='nflId', how='left')

player_points_summary_with_team = player_points_summary_with_team.merge(players_df[['nflId', 'position']], on='nflId', how='left')

player_points_summary_with_team.head()


In [ ]:
# Plotting a histogram for 'total_attempts'
plt.figure(figsize=(10, 6))
plt.hist(player_points_summary_with_team['total_attempts'], bins=100, range=(0, 100), color='blue', alpha=0.7)
plt.title('Histogram of Total Attempts by Players')
plt.xlabel('Total Attempts')
plt.ylabel('Number of Players')
plt.grid(True)
plt.show()

In [ ]:
# Plotting histograms for 'average_points_standardized' and 'average_points_normalized'

plt.figure(figsize=(15, 6))

# Histogram for 'average_points_standardized'
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.hist(player_points_summary_with_team['average_points_standardized'], bins=30, color='orange', alpha=0.7)
plt.title('Histogram of Average Points (Standardized) per Attempt')
plt.xlabel('Average Points (Standardized) per Attempt')
plt.ylabel('Number of Players')
plt.grid(True)

# Histogram for 'average_points_normalized'
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.hist(player_points_summary_with_team['average_points_normalized'], bins=30, color='purple', alpha=0.7)
plt.title('Histogram of Average Points (Normalized) per Attempt')
plt.xlabel('Average Points (Normalized) per Attempt')
plt.ylabel('Number of Players')
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
player_points_summary_with_team.sort_values(by='total_points_standardized', ascending=True).head(10)

In [ ]:
player_points_summary_with_team.sort_values(by='total_points_normalized', ascending=True).head(10)

In [ ]:
# Correcting the data preparation for heatmaps
# For heatmap 1 (by quarters), we sum up each type of tackle event
heatmap1_data = tackles_enhanced.groupby('quarter')[['tackle', 'assist', 'forcedFumble', 'pff_missedTackle']].sum()

# For heatmap 2 (by downs), we sum up each type of tackle event
heatmap2_data = tackles_enhanced.groupby('down')[['tackle', 'assist', 'forcedFumble', 'pff_missedTackle']].sum()

# Plotting the first heatmap (tackle events by quarters)
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap1_data, annot=True, fmt="d", cmap='viridis')
plt.title("Tackle Events by Quarters")
plt.ylabel("Quarter")
plt.xlabel("Tackle Type")
plt.show()

# Plotting the second heatmap (tackle events by downs)
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap2_data, annot=True, fmt="d", cmap='viridis')
plt.title("Tackle Events by Downs")
plt.ylabel("Down")
plt.xlabel("Tackle Type")
plt.show()

### Tackle Events by Quarters

*Observations:*
- Increased tackle events in later quarters, particularly the 4th quarter.
- Missed tackles are somewhat evenly distributed across quarters.
- Assists and forced fumbles are less frequent, with even distribution.

### Tackle Events by Downs

*Observations:*
- Majority of tackles occur on the first and second downs.
- Third down shows fewer tackle events.
- Fourth down has the least number of tackle events.

### Tackle to Missed Tackle Ratios

**Ratios by Quarter:**
- Quarter 1: 5.52
- Quarter 2: 4.53
- Quarter 3: 4.38
- Quarter 4: 4.68
- Overtime (Quarter 5): 5.73

**Ratios by Down:**
- Down 1: 4.59
- Down 2: 4.80
- Down 3: 4.82
- Down 4: 13.90

*Observations:*
- Consistent ratios across quarters with a slight increase in the 1st quarter and overtime.
- Notable outlier in the fourth down, indicating higher tackle success.

In [ ]:
# Creating a new DataFrame for the combined heatmap with quarter and down combinations
tackles_combined = tackles_enhanced.copy()
# Modifying the quarter_down category to follow the format "Q#-D#"
tackles_combined['quarter_down'] = 'Q' + tackles_combined['quarter'].astype(str) + '-D' + tackles_combined['down'].astype(str)

# Regrouping by the modified quarter_down category
heatmap_combined_data = tackles_combined.groupby('quarter_down')[['tackle', 'assist', 'forcedFumble', 'pff_missedTackle']].sum()

# Creating the heatmap with square cells
plt.figure(figsize=(20, 8))
sns.heatmap(heatmap_combined_data.T, annot=True, fmt="d", cmap='plasma', square=True)
plt.title("Tackle Events by Quarter-Down Combination")
plt.ylabel("Tackle Type")
plt.xlabel("Quarter-Down")
plt.show()


### Tackle Events by Quarter-Down Combination

*Observations:*
- **Tackles:** More frequent in early downs, with an increase in the 4th quarter.
- **Missed Tackles:** Evenly distributed, suggesting consistent performance.
- **Forced Fumbles:** Slight increase in the 4th quarter, indicating aggressive play.
- **Assists:** Similar pattern to solo tackles, consistent across game situations.

In [ ]:
# Merging tackles_df with players_df to add the position of the defensive players
tackles_df_with_position = tackles_df.merge(players_df[['nflId', 'position']], on='nflId', how='left')

# Grouping by position and summing up each type of tackle event
heatmap_position_data = tackles_df_with_position.groupby('position')[['tackle', 'assist', 'forcedFumble', 'pff_missedTackle']].sum()

# Creating the heatmap
plt.figure(figsize=(15, 8))
sns.heatmap(heatmap_position_data.T, annot=True, fmt="d", cmap='viridis', square=True)
plt.title("Tackle Events by Defensive Player Position")
plt.ylabel("Tackle Type")
plt.xlabel("Position")
plt.show()

### Average Heights and Weights of Offensive and Defensive Players

- **Offensive Players:**
  - **Average Height:** Approximately 74.64 inches (about 6 feet 2.64 inches)
  - **Average Weight:** Approximately 250.18 pounds

- **Defensive Players:**
  - **Average Height:** Approximately 74.08 inches (about 6 feet 2.08 inches)
  - **Average Weight:** Approximately 252.58 pounds

*Note:*
- These averages provide a general understanding of the physical stature of players in offensive and defensive roles, highlighting that both groups are quite similar in height, with defensive players being slightly heavier on average.


## License

This Notebook is released under the [Apache 2.0 open source license](https://opensource.org/licenses/Apache-2.0).
